# PCA for Dimensionality Reduction

In this notebook we explore Principal Component Analysis (PCA) and how it can be used to interpret high dimensional datasets in lower dimensions. 
To do so, we review concepts from linear algebra to get a fuller understanding of what's going on and why PCA works.
We then walk through running PCA on a small dataset. 

We first review how PCA is defined, what exactly the principal components are, and how PCA allows us to reduce the dimensionality of our dataset in a *good* way. 
We then see how we can use PCA in Python by analyzing a [diabetes dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) provided by NCSU
and what the principal components are. 


Suppose we have a dataset of $n$ samples where each sample is described by $m$ real valued attributes. 
For the purpose of dimensionality reduction, our goal will be to find a matrix $Y$ of size $n \times d$ where $d << m $ and $Y$ approximates $X$ 
(that is, $Y$ still captures the underlying properties of the dataset.)
In this way, the $d$ remaining attributes would be the componenets of $X$ that capture the most relevant information of our dataset, or are its **principal components**.

Before we see how we can compute these componenets, let's consider some scenarios where such a reduction can occur. 
One possibility is a set of attributes within the dataset are the same for each sample - there is no information that these attributes have that allow us to distinguish one sample from another. 
Therefore we could easily remove this set of attributes shrinking the size of $X$.
Typically this is not the case however and there is some variability between sample values for a particular attribute. 
An attribute that has less variability amongst its values may contain less information than an attribute with high variability and therefore could be removed from $X$ to generate $Y$.

Another possiblity is there are some dependency between attributes within the data. 
For example, if we find $x_i = x_j + x_k$ for all samples and attributes $i,j,k$ then attribute $i$ is dependent on $j$ and $k$ and could be removed.
In this way, there is some **correlation** between our attributes, and while a perfect correlation may not always exist due to noisy measurements, we may be able to drop many attributes without losing too much information.

## Principal Components of a Dataset 

Suppose we have a dataset of $n$ samples. Each sample is measured by one of $m$ attributes.
We can interpret this as a matrix $X$ of size $n \times m$  where each row vector corresponds to a sample. 
For the purpose of dimensionality reduction, we would like to find a matrix $Y$ of size $n \times d$ where $d << m $ and $Y$ approximates $X$ 
(that is, $Y$ still captures the underlying properties of the dataset.)
In this way, the $d$ remaining attributes would be the componenets of $X$ that capture the most relevant information, or are the **principal components** of $X$.

Before we see how we can compute these componenets, let's consider some scenarios where such a reduction can occur. 
One possibility is a set of attributes within the dataset are the same for each sample - there is no information that these attributes have that allow us to distinguish one sample from another. 
Therefore we could easily remove this set of attributes shrinking the size of $X$.
Typically this is not the case however and there is some variability between sample values for a particular attribute. 
An attribute that has less variability amongst its values may contain less information than an attribute with high variability and therefore could be removed from $X$ to generate $Y$.

Another possiblity is there are some dependency between attributes within the data. 
For example, if we find $x_i = x_j + x_k$ for all samples and attributes $i,j,k$ then attribute $i$ is dependent on $j$ and $k$ and could be removed.


## Linear Algebra Recap: SVD, Eigenpairs, etc.

To make the steps to PCA clearer, we first review some concepts from linear algebra. 
We do not aim to prove any statements here but encourage the reader to dive deeper into them using resources like
Gilber String's introduction to linear algebra books or Brunton and Kutz's [Data Drive Science & Engineering](https://databookuw.com/databook.pdf) book.

### Means and Variance

Given $n$ samples and $m$ attribute vectors, we can compute what the **mean value** is for each attribute is, and compare each value to how it differs from the mean. 
We can then define what the variance for each attribute is. 
However we can further extend this by considering what the variance is between any two attributes. 
These are combined into a single $m \times m$ matrix called the **covariance** matrix. 

If our dataset is mean centered (the mean rows of $X$ are 0) it follows that the covariance matrix of $X$ is
$$
C = \frac{1}{n-1}X^TX
$$

### Singular Value Decomposition (SVD)

The SVD of a matrix $X$ is a decomposition of $X$ into 3 special matrices:
$$
X = U \Sigma V^T
$$
where 
- $U$ is the left singular value vectors, a $n \times n$ matrix,
- $\Sigma$ is a real, non-negative valued diagonal matrix (zeros off the main diagonal - contains 0s below if $n \geq m$), a $m \times n$ matrix, and
- $V$ is the right singular value vectors, a $m \times m$ matrix.

There are other notable properties are:
1. The order of $u_1, u_2, ..., u_n$ is such that $u_1$ describes the variance in the cols of $X$ more than $u_2, ..., u_n$, $u_2$ describes the variance in the cols of $X$ more than $u_3, ...$, and so on.
2. $U$ and $V$ are unitary matrices such that $U^TU = V^TV = I$ (the identity matrix.)
3. The entries of $\Sigma$ are $\sigma_1, \sigma_2, ...$ and are ordered such that $\sigma_1 \geq \sigma_2 \geq ... \geq 0$.
4. Columns within $U$ and $V$ are orthonormal.

We do not concern ourselves with how the decomposition can be calculated (Python's `numpy` package provides our implementation) only that such a decomposition exists. 

### Eigenvalues and Eigenvectors

For a matrix $A$ of size $n \times n$, the vector $v$ and value $\lambda$ are called eigenvector and eigenvalue if
$$
Av = \lambda v
$$
A key property of eigenvectors are vectors which are only changes by a scalar value whenever a linear transformation is applied to them.

If we stack all the eigenvectors together into a $n \times n$ matrix $V$ we have
$$
AV = V \Lambda
$$
where $\Lambda$ is a diagonal matrix with matrix filled with eigenvalues.

## SVD and PCA

We now see how the SVD of our dataset relates to its principal components. 
First we mean center our dataset. We then compute $X^TX$ using the SVD of $X$
$$\begin{aligned}
X^TX &= (U \Sigma V^T)^T (U \Sigma V^T) \\
&= V \Sigma^T U^T U \Sigma V^T \\
&= V \Sigma^T \Sigma V^T \\
&= V D V^T
\end{aligned}$$

where $D = \Sigma^T \Sigma$ is a diagonal with squares of singular values and making note of $U$ being a unitary matrix. 

Now we compute $(X^TX)V$ using the above

$$\begin{aligned}
(X^TX)V &= (V D V^T) V \\
(X^TX)V &= V D
\end{aligned}$$
as $V$ is a unitary matrix. 

We now observe that $V$ is the eigenvectors of, not our dataset $X$, but $X^TX$ and that $D$ contains our eigenvalues (which are $\sigma_1^2, \sigma_2^2, ...$).
But since our dataset is mean centered, we know that $X^TX = (n-1)C$ our dataset's covariance matrix scaled by $n-1$. 


In [4]:
from sklearn.datasets import load_diabetes
dia = load_diabetes()


In [8]:
dia.data.shape

(442, 10)

In [15]:
dia.target.shape

(442,)

In [19]:
dia.data[:, 0].min()

-0.1072256316073538

In [20]:
dia.data[:5]

array([[ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, -0.00567042, -0.04559945,
        -0.03419447, -0.03235593, -0.00259226,  0.00286131, -0.02593034],
       [-0.08906294, -0.04464164, -0.01159501, -0.03665608,  0.01219057,
         0.02499059, -0.03603757,  0.03430886,  0.02268774, -0.00936191],
       [ 0.00538306, -0.04464164, -0.03638469,  0.02187239,  0.00393485,
         0.01559614,  0.00814208, -0.00259226, -0.03198764, -0.04664087]])